## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)

## AG Queries

> - Check role (Query 1)
>     - Database States(Query 2)
>     - Determine when nodes have chaneged roles(Query 3)
>     - Last 5 seconds health status (Query 4)
>     - Query Always on Health extended event(Query 5)
>     - Availability info, database, roles, state, recovery times, redo information (Query 6)
>     - Overview of AG health - Glenn Berry Script- (Query 7)
>     - History State change- (Query 8)
>     - Listener Info
>         - Listener Info- Port etc (Query 9.1)
>         - Listener Info IP Address- (Query 9.2)
>         - Listener Info IP TCP listener States- (Query 9.3)
>     - Name of replica Nodes (Query 11)

In [ ]:
/* Check role*/
IF EXISTS(SELECT Name from sysobjects where name ='availability_replicas')
SELECT replica_server_name
	,role_desc
FROM master.[sys].[availability_replicas] ar
JOIN master.sys.dm_hadr_availability_replica_states rs ON ar.replica_id = rs.replica_id

/* Current role */
SELECT role_desc,* FROM sys.dm_hadr_availability_replica_states WHERE is_local=1 


>   Database States- (Query 2)

In [ ]:
/* Database states*/
SELECT
    dc.database_name,
    d.synchronization_health_desc,
    d.synchronization_state_desc,
    d.database_state_desc
From
    sys.dm_hadr_database_replica_states d
    JOIN sys.availability_databases_cluster dc ON d.group_database_id = dc.group_database_id
    AND d.is_local = 1

> Determine when nodes have chaneged roles- (Query 3)

In [ ]:
/* Script to determine failover times in Availability Group */

;WITH cte_HADR AS (SELECT object_name, CONVERT(XML, event_data) AS data
FROM sys.fn_xe_file_target_read_file('AlwaysOn*.xel', null, null, null)
WHERE object_name = 'error_reported'
)

SELECT data.value('(/event/@timestamp)[1]','datetime') AS [timestamp],
       data.value('(/event/data[@name=''error_number''])[1]','int') AS [error_number],
       data.value('(/event/data[@name=''message''])[1]','varchar(max)') AS [message]
FROM cte_HADR
WHERE data.value('(/event/data[@name=''error_number''])[1]','int') = 1480
ORDER BY data.value('(/event/@timestamp)[1]','datetime')  desc

> Last 5 seconds diagnostics- (Query 4)

In [ ]:
/* Last 5 seconds dignostics */
sp_server_diagnostics 10

> Query Always on Health extended event- (Query 5)

In [ ]:
/* ALways on Health extended event */
DECLARE @FileName NVARCHAR(4000)
SELECT @FileName = target_data.value('(EventFileTarget/File/@name)[1]','nvarchar(4000)')
FROM (
        SELECT
            CAST(target_data AS XML) target_data
            FROM sys.dm_xe_sessions s
            JOIN sys.dm_xe_session_targets t
                ON s.address = t.event_session_address
            WHERE s.name = N'AlwaysOn_health'
    ) ft
 
SELECT
    XEData.value('(event/@timestamp)[1]','datetime2(3)') AS event_timestamp,
    XEData.value('(event/data[@name="error_number"]/value)[1]', 'int') AS error_number,
    XEData.value('(event/data[@name="message"]/value)[1]', 'varchar(max)') AS Message,
	XEData.value('(event/@name)[1]', 'varchar(max)') AS Name

FROM (
        SELECT CAST(event_data AS XML) XEData, *
        FROM sys.fn_xe_file_target_read_file(@FileName, NULL, NULL, NULL)
        --WHERE object_name = 'error_reported'
        ) event_data
--WHERE XEData.value('(event/data[@name="error_number"]/value)[1]', 'int')  IN ( 35264, 35265)
    ORDER BY event_timestamp DESC;


> Availability info, database, roles, state, recovery times, redo information- (Query 6)

In [ ]:
/* Availability info, database, roles, state, recovery times, redo information */
--https://sqlundercover.com/2017/09/19/7-ways-to-query-always-on-availability-groups-using-sql/
SET NOCOUNT ON;

DECLARE @AGname NVARCHAR(128);

DECLARE @SecondaryReplicasOnly BIT;

SET @AGname = NULL --N'LiveAG1'; --SET AGname for a specific AG for SET to NULL for ALL AG's

IF OBJECT_ID('TempDB..#tmpag_availability_groups') IS NOT NULL
    DROP TABLE [#tmpag_availability_groups];

SELECT *
INTO [#tmpag_availability_groups]
FROM [master].[sys].[availability_groups];

IF (
       @AGname IS NULL
       OR EXISTS
(
    SELECT [name]
    FROM [#tmpag_availability_groups]
    WHERE [name] = @AGname
)
   )
BEGIN

    IF OBJECT_ID('TempDB..#tmpdbr_availability_replicas') IS NOT NULL
        DROP TABLE [#tmpdbr_availability_replicas];

    IF OBJECT_ID('TempDB..#tmpdbr_database_replica_cluster_states') IS NOT NULL
        DROP TABLE [#tmpdbr_database_replica_cluster_states];

    IF OBJECT_ID('TempDB..#tmpdbr_database_replica_states') IS NOT NULL
        DROP TABLE [#tmpdbr_database_replica_states];

    IF OBJECT_ID('TempDB..#tmpdbr_database_replica_states_primary_LCT') IS NOT NULL
        DROP TABLE [#tmpdbr_database_replica_states_primary_LCT];

    IF OBJECT_ID('TempDB..#tmpdbr_availability_replica_states') IS NOT NULL
        DROP TABLE [#tmpdbr_availability_replica_states];

    SELECT [group_id],
           [replica_id],
           [replica_server_name],
           [availability_mode],
           [availability_mode_desc]
    INTO [#tmpdbr_availability_replicas]
    FROM [master].[sys].[availability_replicas];

    SELECT [replica_id],
           [group_database_id],
           [database_name],
           [is_database_joined],
           [is_failover_ready]
    INTO [#tmpdbr_database_replica_cluster_states]
    FROM [master].[sys].[dm_hadr_database_replica_cluster_states];

    SELECT *
    INTO [#tmpdbr_database_replica_states]
    FROM [master].[sys].[dm_hadr_database_replica_states];

    SELECT [replica_id],
           [role],
           [role_desc],
           [is_local]
    INTO [#tmpdbr_availability_replica_states]
    FROM [master].[sys].[dm_hadr_availability_replica_states];

    SELECT [ars].[role],
           [drs].[database_id],
           [drs].[replica_id],
           [drs].[last_commit_time]
    INTO [#tmpdbr_database_replica_states_primary_LCT]
    FROM [#tmpdbr_database_replica_states] AS [drs]
        LEFT JOIN [#tmpdbr_availability_replica_states] [ars]
            ON [drs].[replica_id] = [ars].[replica_id]
    WHERE [ars].[role] = 1;

    SELECT [AG].[name] AS [AvailabilityGroupName],
           [AR].[replica_server_name] AS [AvailabilityReplicaServerName],
           [dbcs].[database_name] AS [AvailabilityDatabaseName],
           ISNULL([dbcs].[is_failover_ready], 0) AS [IsFailoverReady],
           ISNULL([arstates].[role_desc], 3) AS [ReplicaRole],
           [AR].[availability_mode_desc] AS [AvailabilityMode],
           CASE [dbcs].[is_failover_ready]
               WHEN 1 THEN
                   0
               ELSE
                   ISNULL(DATEDIFF([ss], [dbr].[last_commit_time], [dbrp].[last_commit_time]), 0)
           END AS [EstimatedDataLoss_(Seconds)],
           ISNULL(   CASE [dbr].[redo_rate]
                         WHEN 0 THEN
                             -1
                         ELSE
                             CAST([dbr].[redo_queue_size] AS FLOAT) / [dbr].[redo_rate]
                     END,
                     -1
                 ) AS [EstimatedRecoveryTime_(Seconds)],
           ISNULL([dbr].[is_suspended], 0) AS [IsSuspended],
           ISNULL([dbr].[suspend_reason_desc], '-') AS [SuspendReason],
           ISNULL([dbr].[synchronization_state_desc], 0) AS [SynchronizationState],
           ISNULL([dbr].[last_received_time], 0) AS [LastReceivedTime],
           ISNULL([dbr].[last_redone_time], 0) AS [LastRedoneTime],
           ISNULL([dbr].[last_sent_time], 0) AS [LastSentTime],
           ISNULL([dbr].[log_send_queue_size], -1) AS [LogSendQueueSize],
           ISNULL([dbr].[log_send_rate], -1) AS [LogSendRate_KB/S],
           ISNULL([dbr].[redo_queue_size], -1) AS [RedoQueueSize_KB],
           ISNULL([dbr].[redo_rate], -1) AS [RedoRate_KB/S],
           ISNULL(   CASE [dbr].[log_send_rate]
                         WHEN 0 THEN
                             -1
                         ELSE
                             CAST([dbr].[log_send_queue_size] AS FLOAT) / [dbr].[log_send_rate]
                     END,
                     -1
                 ) AS [SynchronizationPerformance],
           ISNULL([dbr].[filestream_send_rate], -1) AS [FileStreamSendRate],
           ISNULL([dbcs].[is_database_joined], 0) AS [IsJoined],
           [arstates].[is_local] AS [IsLocal],
           ISNULL([dbr].[last_commit_lsn], 0) AS [LastCommitLSN],
           ISNULL([dbr].[last_commit_time], 0) AS [LastCommitTime],
           ISNULL([dbr].[last_hardened_lsn], 0) AS [LastHardenedLSN],
           ISNULL([dbr].[last_hardened_time], 0) AS [LastHardenedTime],
           ISNULL([dbr].[last_received_lsn], 0) AS [LastReceivedLSN],
           ISNULL([dbr].[last_redone_lsn], 0) AS [LastRedoneLSN]
    FROM [#tmpag_availability_groups] AS [AG]
        INNER JOIN [#tmpdbr_availability_replicas] AS [AR]
            ON [AR].[group_id] = [AG].[group_id]
        INNER JOIN [#tmpdbr_database_replica_cluster_states] AS [dbcs]
            ON [dbcs].[replica_id] = [AR].[replica_id]
        LEFT OUTER JOIN [#tmpdbr_database_replica_states] AS [dbr]
            ON [dbcs].[replica_id] = [dbr].[replica_id]
               AND [dbcs].[group_database_id] = [dbr].[group_database_id]
        LEFT OUTER JOIN [#tmpdbr_database_replica_states_primary_LCT] AS [dbrp]
            ON [dbr].[database_id] = [dbrp].[database_id]
        INNER JOIN [#tmpdbr_availability_replica_states] AS [arstates]
            ON [arstates].[replica_id] = [AR].[replica_id]
    WHERE [AG].[name] = ISNULL(@AGname, [AG].[name])
    ORDER BY [AvailabilityReplicaServerName] ASC,
             [AvailabilityDatabaseName] ASC;

END;
ELSE
BEGIN
    RAISERROR('Invalid AG name supplied, please correct and try again', 12, 0);
END;


> Overview of AG health - Glenn Berry Script- (Query 7)

In [ ]:
/* overview of AG health and status -Glenn Berry script */
--From Glenn Berry script	SQL 2019
-- Good overview of AG health and status (Query 17) (AlwaysOn AG Status)
SELECT ag.name AS [AG Name], ar.replica_server_name, ar.availability_mode_desc, adc.[database_name], 
       drs.is_local, drs.is_primary_replica, drs.synchronization_state_desc, drs.is_commit_participant, 
	   drs.synchronization_health_desc, drs.recovery_lsn, drs.truncation_lsn, drs.last_sent_lsn, 
	   drs.last_sent_time, drs.last_received_lsn, drs.last_received_time, drs.last_hardened_lsn, 
	   drs.last_hardened_time, drs.last_redone_lsn, drs.last_redone_time, drs.log_send_queue_size, 
	   drs.log_send_rate, drs.redo_queue_size, drs.redo_rate, drs.filestream_send_rate, 
	   drs.end_of_log_lsn, drs.last_commit_lsn, drs.last_commit_time, drs.database_state_desc 
FROM sys.dm_hadr_database_replica_states AS drs WITH (NOLOCK)
INNER JOIN sys.availability_databases_cluster AS adc WITH (NOLOCK)
ON drs.group_id = adc.group_id 
AND drs.group_database_id = adc.group_database_id
INNER JOIN sys.availability_groups AS ag WITH (NOLOCK)
ON ag.group_id = drs.group_id
INNER JOIN sys.availability_replicas AS ar WITH (NOLOCK)
ON drs.group_id = ar.group_id 
AND drs.replica_id = ar.replica_id
ORDER BY ag.name, ar.replica_server_name, adc.[database_name] OPTION (RECOMPILE);

### Q8 History Change States

In [ ]:
/* availability_replica_state_change */
--https://dba.stackexchange.com/questions/76016/how-to-check-history-of-primary-node-in-an-availability-group
DECLARE @xel_path VARCHAR(1024);
DECLARE @utc_adjustment INT = datediff(hour, getutcdate(), getdate());
	-------------------------------------------------------------------------------
	------------------- target event_file path retrieval --------------------------
	-------------------------------------------------------------------------------
	;

WITH target_data_cte
AS (
	SELECT target_data = convert(XML, target_data)
	FROM sys.dm_xe_sessions s
	INNER JOIN sys.dm_xe_session_targets st ON s.address = st.event_session_address
	WHERE s.NAME = 'alwayson_health'
		AND st.target_name = 'event_file'
	)
	,full_path_cte
AS (
	SELECT full_path = target_data.value('(EventFileTarget/File/@name)[1]', 'varchar(1024)')
	FROM target_data_cte
	)
SELECT @xel_path = left(full_path, len(full_path) - charindex('\', reverse(full_path))) + '\AlwaysOn_health*.xel'
FROM full_path_cte;
	-------------------------------------------------------------------------------
	------------------- replica state change events -------------------------------
	-------------------------------------------------------------------------------
	;

WITH state_change_data
AS (
	SELECT object_name
		,event_data = convert(XML, event_data)
	FROM sys.fn_xe_file_target_read_file(@xel_path, NULL, NULL, NULL)
	)
SELECT object_name
	,event_timestamp = dateadd(hour, @utc_adjustment, event_data.value('(event/@timestamp)[1]', 'datetime'))
	,ag_name = event_data.value('(event/data[@name = "availability_group_name"]/value)[1]', 'varchar(64)')
	,previous_state = event_data.value('(event/data[@name = "previous_state"]/text)[1]', 'varchar(64)')
	,current_state = event_data.value('(event/data[@name = "current_state"]/text)[1]', 'varchar(64)')
FROM state_change_data
WHERE object_name = 'availability_replica_state_change'
ORDER BY event_timestamp DESC;


>   - Listener Info- (Query 9.1)

In [ ]:
/* Listener info */
SELECT * FROM sys.availability_group_listeners


> - Listener Info IP Address- (Query 9.2)

In [ ]:
/*Returns a row for every conformant virtual IP address that is currently online for an availability group listener.*/
select * from sys.availability_group_listener_ip_addresses


> - Listener Info IP TCP listener States- (Query 9.3)

In [ ]:
/*-Returns a row containing dynamic-state information for each TCP listener.*/
SELECT * FROM sys.dm_tcp_listener_states


>    -  Check if databse is read/write or readonly- (Query 10.1)

In [ ]:
/*Check if read/write readonly*/
SELECT DATABASEPROPERTYEX(DB_NAME(), 'Updateability')


 >  -  Check if databse is read/write or readonly- (Query 10.1)

In [ ]:
/* */
SELECT	AV.name AS AVGName
	, AVGLis.dns_name AS ListenerName
	, AVGLis.ip_configuration_string_from_cluster AS ListenerIP
FROM	sys.availability_group_listeners AVGLis
INNER JOIN sys.availability_groups AV on AV.group_id = AVGLis.group_id


 >  -  Read only urls (Query 10.2)

In [ ]:
 >  -  Read only routing infomation (Query 10.3)SELECT replica_server_name
	, read_only_routing_url
	, secondary_role_allow_connections_desc
FROM sys.availability_replicas


 <span style="color: #777777;">&gt;</span>  <span style="color: #777777;">-</span> <span style="color: #4b69c6;">&nbsp;&nbsp;Read&nbsp;only&nbsp;urls&nbsp;(Query&nbsp;10.2)</span>

In [ ]:
/* Read only routing information */
SELECT	  AVGSrc.replica_server_name AS SourceReplica		
		, AVGRepl.replica_server_name AS ReadOnlyReplica
		, AVGRepl.read_only_routing_url AS RoutingURL
		, AVGRL.routing_priority AS RoutingPriority
FROM sys.availability_read_only_routing_lists AVGRL
INNER JOIN sys.availability_replicas AVGSrc ON AVGRL.replica_id = AVGSrc.replica_id
INNER JOIN sys.availability_replicas AVGRepl ON AVGRL.read_only_replica_id = AVGRepl.replica_id
INNER JOIN sys.availability_groups AV ON AV.group_id = AVGSrc.group_id
ORDER BY SourceReplica

 >  -  Name of replica Nodes (Query 11)

In [ ]:
    /* Node Names */
    SELECT 
           [replica_server_name]
    FROM [master].[sys].[availability_replicas];

## [Home](file:///C:/SQLScriptsLibrary/Home.ipynb)